In [1]:
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2025, 2, 21)

In [3]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-02-21
yesterday: 2025-02-20


In [5]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\Downloads\Datasets
dts_path = os.path.join(user_path, "Downloads", "Datasets")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [7]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### Tables in the process

In [10]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [12]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,ASK,57,2023-02-16,"4,500",31.00,1081,1242,2025-02-17,7.35,68.47,4.79,"33,001.74",732,"-106,807.24",-76.40%,-38.09%,-81,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [15]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)
name

'ASK'

In [17]:
sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1242



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,ASK,1242,57,2023-02-16,"4,500",31.00,288.77,20.21,Sold,"139,808.98",DTD,None


In [19]:
transaction, name, qty, sell_price, buy_price

('S', 'ASK', -4500, 7.35, 31.0)

### In case of buy, retreive buy id from latest buy record (id)

In [12]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,MCS,1316,43,2025-02-21,"6,000",7.30,90.67,6.35,Active,"43,897.02",HD,None


In [14]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'MCS', 6000, 7.3)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [21]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'ASK'



,name,date,shares,price,active,period,grade
0,ASK,2023-02-16,"6,000",26.25,1,1,C1


In [25]:
print(transaction, name, qty, buy_price)

S ASK -4500 31.0


In [33]:
new_qty = buy['shares'] + qty
print(new_qty.iloc[0])

0


In [ ]:
def update_buy(const, name, new_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty.iloc[0])
    
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,
        'name': name
    })

    return f"Records updated = {rp.rowcount}"

In [ ]:
# Example usage
update_buy(const, name, new_qty)

'Records updated = 1'

In [37]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'ASK'



,name,date,shares,price,active,period,grade
0,ASK,2023-02-16,0,26.25,1,1,C1


In [41]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM buy
    WHERE name = :name AND volbuy = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 1


### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [43]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'ASK'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,ASK,1.1600,0.0000,0.0000,0.0000,1.1600,"6,000",2024-02-29,2024-04-30,,0


In [47]:
new_qty.iloc[0]

0

In [53]:
def update_dividend(const, name, new_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty.iloc[0])
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,
        'name': name
    })
    return f"Records updated = {rp.rowcount}"

In [55]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [57]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'ASK'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,ASK,1.1600,0.0000,0.0000,0.0000,1.1600,0,2024-02-29,2024-04-30,,0


In [59]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM dividend
    WHERE name = :name AND shares = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 1


### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [85]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stock = pd.read_sql(sql, conlite)
stock.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'ASK'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,102,ASK,0.00,0.00,X,10.00,26.25,0.00,0.00,26.25,"1,500",-4,4,"6,000",0.000000,0.000000,RD60Pct,SET


In [95]:
# Extract scalar values from Pandas Series (if applicable)
new_qty_value = new_qty.iloc[0] if hasattr(new_qty, 'iloc') else new_qty
cost_value = cost.iloc[0] if hasattr(cost, 'iloc') else cost

# Print the values
print(name, transaction, new_qty_value, cost_value)
print(new_buy_target, new_sell_target, new_sell_qty)

ASK S 0 15.4
10.0 26.25 1500


In [87]:
print("Raw cost value before extraction:", cost)

Raw cost value before extraction: 15.4


In [ ]:
def update_stock(conlite, name, new_qty, new_buy_target, new_sell_target, new_sell_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty.iloc[0])  
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty, 
            buy_target = :new_buy_target,
            sell_target = :new_sell_target,    
            qty = :new_sell_qty
        WHERE name = :name
    """)    
    # Execute the query with parameters
    rp = conlite.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'new_buy_target': new_buy_target,
        'new_sell_target': new_sell_target,     
        'new_sell_qty': new_sell_qty,
        'name': name
    })
    return f"Records updated = {rp.rowcount}"

In [ ]:
# Example usage
update_stock(conlite, name, new_qty, new_buy_target, new_sell_target, new_sell_qty)

'Records updated = 1'

In [99]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'ASK'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,102,ASK,0.00,0.00,X,10.00,26.25,0.00,0.00,26.25,"1,500",-4,4,0,0.000000,0.000000,RD60Pct,SET


In [103]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM stocks
    WHERE name = :name AND available_qty = 0;
""")

# Execute the query with the correct parameter dictionary
result = conlite.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 1


In [105]:
conlite.commit()

### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

### =====================================================

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [16]:
sql = """
SELECT *, volbuy * price AS cost_amt 
FROM buy
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy['date'] = pd.to_datetime(buy['date'])
buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
buy.rename(columns={'volbuy':'shares'}, inplace = True)
buy['shares'] = buy['shares'].astype('int64')
buy


SELECT *, volbuy * price AS cost_amt 
FROM buy
WHERE name = 'MCS'



,name,date,shares,price,active,dividend,period,grade,cost_amt
0,MCS,2016-09-20,75000,15.4,1,0.52,2,A3,1155000.0


In [18]:
print(transaction, name, qty, buy_price)

B MCS 6000 7.3


In [20]:
new_unit_cost = buy['cost_amt'].iloc[0] + (qty * buy_price)
new_qty = buy['shares'].iloc[0] + qty
print(new_unit_cost)
print(new_qty)
new_unit_cost = new_unit_cost / new_qty
print(new_unit_cost)

1198800.0
81000
14.8


### In case there is the same stock name in port

In [27]:
def update_buy(const, name, new_qty, new_unit_cost):
    # Extract scalar values from Pandas Series
    new_qty_value = new_qty
    new_unit_cost_value = new_unit_cost
    
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty, 
            price = :new_unit_cost
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'new_unit_cost': new_unit_cost_value,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {rp.rowcount}"

In [29]:
# Example usage
update_buy(const, name, new_qty, new_unit_cost)

'Records updated = 1'

In [31]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'MCS'



,name,date,shares,price,active,dividend,period,grade
0,MCS,2016-09-20,"81,000",14.80,1,0.5200,2,A3


### In case of totally new stock

In [11]:
rcd = ['TVO', '2024-11-06', 4000, 24.90, 0, 0, 1, 0, 4,'A2']
rcd

['TVO', '2024-11-06', 4000, 24.9, 0, 0, 1, 0, 4, 'A2']

In [12]:
sqlIns = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [13]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [15]:
name = 'TVO'

In [16]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'TVO'



,name,date,shares,price,active,dividend,period,grade
0,TVO,2024-11-06,"4,000",24.90,1,0.0000,4,A2


### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [33]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'MCS' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,MCS,0.5000,0.0000,0.2000,0.0000,0.7000,"75,000",2025-04-18,2025-05-07,,1


In [35]:
new_qty = dividend['shares'].iloc[0] + qty
print(new_qty)

81000


In [37]:
def update_dividend(const, name, new_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = new_qty  
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    rp = const.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {rp.rowcount}"

In [39]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [41]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'MCS' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,MCS,0.5000,0.0000,0.2000,0.0000,0.7000,"81,000",2025-04-18,2025-05-07,,1


### In case of totally new stock

In [15]:
rcd = ['TVO', 0.25, 0, 0.33, 0, 0.58, 1000, '2023-08-30', '2023-09-14', '', 1]
rcd

['TOA', 0.25, 0, 0.33, 0, 0.58, 1000, '2023-08-30', '2023-09-14', '', 1]

In [15]:
sqlIns = """
INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [16]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [17]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'TOA' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,TOA,0.2500,0.0000,0.3300,0.0000,0.5800,"1,000",2023-08-30,2023-09-14,,1


### End of Create dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [43]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stock = pd.read_sql(sql, conlite)
stock


SELECT * 
FROM stocks 
WHERE name = 'MCS'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,114,MCS,0.0,0.0,B,5.8,12.9,0.0,0.0,15.4,5000,-4,4,75000,0.0,0.0,RD60Pct,SET


In [47]:
#
new_buy_target = 6.3
new_sell_target = 14.80
new_buy_qty = 4_000
print(transaction, name, qty, buy_price, new_buy_target, new_sell_target, new_buy_qty) 

B MCS 6000 7.3 6.3 14.8 4000


In [51]:
print(new_qty)

81000


In [55]:
print(new_unit_cost)

14.8


In [57]:
def update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty):
    # Extract scalar values from Pandas Series
    new_qty_value = int(new_qty)
    new_unit_cost_value = float(new_unit_cost)   
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty, 
            cost = :new_unit_cost,
            buy_target = :new_buy_target,
            sell_target = :new_sell_target,    
            qty = :new_buy_qty
        WHERE name = :name
    """)    
    # Execute the query with parameters
    rp = conlite.execute(sqlUpd, {
        'new_qty': new_qty_value,  # Use the scalar value
        'new_unit_cost': new_unit_cost_value, # Use the scalar value 
        'new_buy_target': new_buy_target,
        'new_sell_target': new_sell_target,     
        'new_buy_qty': new_buy_qty,
        'name': name
    })
    return f"Records updated = {rp.rowcount}"

In [59]:
# Example usage
update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty)

'Records updated = 1'

In [61]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stock = pd.read_sql(sql, conlite)
stock


SELECT * 
FROM stocks 
WHERE name = 'MCS'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,114,MCS,0.0,0.0,B,6.3,14.8,0.0,0.0,14.8,4000,-4,4,81000,0.0,0.0,RD60Pct,SET


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process

### =================================================

In [65]:
#price_date = '2025-02-14'
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '%s' \
ORDER BY period, buy.name"
sql = sql % (yesterday)
print(sql)

SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2025-02-20' ORDER BY period, buy.name


In [67]:
output = pd.read_sql(sql, const)
output

,period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
0,1,C1,AH,2023-06-08,"1,200",37.00,14.00,"44,400.00","16,800.00","-27,600.00",-62.16
1,1,C1,ASP,2022-03-07,"30,000",3.80,2.30,"114,000.00","69,000.00","-45,000.00",-39.47
2,1,C3,IVL,2022-08-29,"7,200",40.00,22.10,"288,000.00","159,120.00","-128,880.00",-44.75
3,1,C3,ORI,2022-04-07,"60,000",9.15,3.14,"549,000.00","188,400.00","-360,600.00",-65.68
4,1,C4,PTTGC,2021-03-17,"6,000",64.75,19.20,"388,500.00","115,200.00","-273,300.00",-70.35
5,1,C2,SCC,2021-09-20,600,405.00,143.00,"243,000.00","85,800.00","-157,200.00",-64.69
6,1,A2,SINGER,2023-01-19,"6,000",24.80,6.00,"148,800.00","36,000.00","-112,800.00",-75.81
7,1,C2,STA,2021-06-15,"10,000",30.00,16.70,"300,000.00","167,000.00","-133,000.00",-44.33
8,1,C1,TMT,2021-08-16,"36,000",10.20,3.00,"367,200.00","108,000.00","-259,200.00",-70.59
9,1,C1,TOA,2023-09-06,"1,000",26.00,12.20,"26,000.00","12,200.00","-13,800.00",-53.08


In [69]:
file_name = 'portfolio.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [71]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\portfolio.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\portfolio.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\portfolio.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\portfolio.csv


In [73]:
output.to_csv(output_file, header=True, index=False)
output.to_csv(icd_file, header=True, index=False)
output.to_csv(god_file, header=True, index=False)
output.to_csv(osd_file, header=True, index=False)

In [75]:
file_name = '035-portfolio.xlsx'
xsl_file = os.path.join(xsl_path, file_name)

output.to_excel(xsl_file, index=False)